In [1]:
import requests

requests.get("http://naver.com").text

'<!doctype html>\n<html lang="ko">\n<head>\n<meta charset="utf-8">\n<meta http-equiv="Content-Script-Type" content="text/javascript">\n<meta http-equiv="Content-Style-Type" content="text/css">\n<meta http-equiv="X-UA-Compatible" content="IE=edge">\n<meta name="viewport" content="width=960">\n<meta name="apple-mobile-web-app-title" content="NAVER" />\n<meta property="og:title" content="네이버 메인">\n<meta property="og:url" content="http://www.naver.com/">\n<meta property="og:image" content="http://static.naver.net/www/mobile/edit/2016/0705/mobile_212852414260.png">\n<meta property="og:description" content="네이버 메인에서 다양한 정보와 유용한 컨텐츠를 만나 보세요">\n<meta name="twitter:card" content="summary">\n<meta name="twitter:title" content="네이버 메인">\n<meta name="twitter:url" content="http://www.naver.com/">\n<meta name="twitter:image" content="http://static.naver.net/www/mobile/edit/2016/0705/mobile_212852414260.png">\n<meta name="twitter:description" content="네이버 메인에서 다양한 정보와 유용한 컨텐츠를 만나 보세요">\n<link rel="sh

In [8]:
html_url = "http://comic.naver.com/webtoon/detail.nhn?titleId=675554&no=195&weekday=mon"
image_url = "http://imgcomic.naver.net/webtoon/675554/195/20161125144754_446f1bf713b3a84cc75a67b738578d96_IMAG01_1.JPG"

import requests
html = requests.get(html_url).text
open("dump.html", "wt", encoding="utf8").write(html)

image_data = requests.get(image_url, headers={'referer': html_url}).content
open("dump.jpg", "wb").write(image_data)

242857

In [6]:
!ls

README.md
dump.html
dump.jpg
main01.py
sample.csv
test.html
test.log
파이썬 강의 #01.ipynb
파이썬 강의 #02
파이썬 강의 #02.ipynb
파이썬 강의 #03.ipynb
파이썬 강의 #04
파이썬 강의 #04.ipynb
파이썬 강의 #05
파이썬 강의 #05.ipynb


In [17]:
# 국회의원 목록
url = "http://www.assembly.go.kr/assm/memact/congressman/memCond/memCondListAjax.do?currentPage=1&rowPerPage=300"

import requests
html = requests.get(url).text

'''
기본 구조
class="memberna_list"
    <a href="javascript:jsMemPop('9770276')" title="강길부의원정보 새창에서 열림">강길부</a>
    <a href="javascript:jsMemPop('9770933')" title="강병원의원정보 새창에서 열림">강병원</a>
    <a href="javascript:jsMemPop('9771036')" title="강석진의원정보 새창에서 열림">강석진</a>
'''

import re
from bs4 import BeautifulSoup

soup = BeautifulSoup(html, 'html.parser')
for idx, tag in enumerate(soup.select('.memberna_list a[href*=jsMemPop]'), 1):
    # print(tag)
    js = tag['href']
    matched = re.match(r'javascript:jsMemPop\(\'(\d+)\'\)', js)
    if matched:
        mem_code = matched.group(1)
        mem_name = tag.text
        print(idx, mem_code, mem_name)

1 9770276 강길부
2 9770933 강병원
3 9771036 강석진
4 9770512 강석호
5 9770279 강창일
6 9771054 강효상
7 9771007 강훈식
8 9770708 경대수
9 9770931 고용진
10 9770961 곽대훈
11 9770956 곽상도
12 9771066 권미혁
13 9771004 권석창
14 9770679 권성동
15 9770977 권은희
16 9770998 권칠승
17 9770937 금태섭
18 9770927 기동민
19 9771033 김경수
20 9770970 김경진
21 9770712 김경협
22 9770713 김관영
23 9770523 김광림
24 9771011 김광수
25 9771044 김규환
26 9770716 김기선
27 9770719 김도읍
28 9770295 김동철
29 9770997 김두관
30 9770721 김명연
31 9770895 김무성
32 9770723 김민기
33 9770983 김병관
34 9770941 김병기
35 9770984 김병욱
36 9770960 김부겸
37 9771077 김삼화
38 9770725 김상훈
39 9770527 김상희
40 9771024 김석기
41 9770930 김선동
42 9771065 김성수
43 9770942 김성식
44 9770985 김성원
45 9770727 김성찬
46 9770974 김성태
47 9771046 김성태
48 9770534 김세연
49 9771075 김수민
50 9771053 김순례
51 9771049 김승희
52 9770538 김영우
53 9770975 김영주
54 9770981 김영진
55 9770949 김영춘
56 9770934 김영호
57 9770542 김용태
58 9770310 김재경
59 9770993 김정우
60 9771023 김정재
61 9770316 김정훈
62 9771083 김종대
63 9771009 김종민
64 9771048 김종석
65 9771057 김종인
66 9770738 김종태
67 9771016 김종회
68 9

## Selenium

In [18]:
python_query_url = "https://search.naver.com/search.naver?sm=tab_hty.top&where=nexearch&oquery=%ED%8C%8C%EC%9D%B4%EC%8D%AC&ie=utf8&query=%ED%8C%8C%EC%9D%B4%EC%8D%AC"

import requests
html = requests.get(python_query_url).text
open("python_query.html", "wt", encoding="utf8").write(html)

256491

In [20]:
# 네이버 검색 수행

import os
from contextlib import contextmanager
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

@contextmanager
def chrome():
    browser = None
    try:
        browser = webdriver.Chrome('./mac64_chromedriver')  # FIXME: 각자 경로에 맞게 수정해주세요.
        yield browser
    finally:
        if browser:
            browser.close()

def main():
    with chrome() as browser:
        browser.get('http://naver.com')

        element = browser.find_element_by_name('query')
        element.send_keys('파이썬')
        element.send_keys(Keys.RETURN)

        delay = 10
        try:
            WebDriverWait(browser, delay).until(
                EC.presence_of_element_located((By.ID, 'main_pack'))
            )
        except TimeoutException:
            print('timeout')

        print('page loaded !!!')

if __name__ == '__main__':
    main()

page loaded !!!


In [21]:
input('너의 이름이 무엇이냐?')

너의 이름이 무엇이냐?이진석입니다.


'이진석입니다.'

In [22]:
input('너의 암호가 무엇이냐?')

너의 암호가 무엇이냐?1234


'1234'

In [23]:
from getpass import getpass
getpass('너의 암호가 무엇이냐?')

너의 암호가 무엇이냐?········


'1234'

In [ ]:
# 네이버 이메일 목록 수집

import os
import sys
from time import sleep
from getpass import getpass
from bs4 import BeautifulSoup
from contextlib import contextmanager
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


@contextmanager
def chrome():
    browser = None
    try:
        browser = webdriver.Chrome('./chromedriver')  # FIXME: 각자 환경에 맞게 수정해주세요.
        yield browser
    finally:
        if browser:
            browser.close()


def wait(browser, delay, by, target):
    try:
        WebDriverWait(browser, delay).until(
            EC.presence_of_element_located((by, target))
        )
    except TimeoutException:
        print('timeout')
        sys.exit(1)


def main(user_id, user_pw):
    with chrome() as browser:
        browser.get('https://nid.naver.com/nidlogin.login')

        element = browser.find_element_by_id('id')
        element.send_keys(user_id)

        element = browser.find_element_by_id('pw')
        element.send_keys(user_pw)

        element.send_keys(Keys.RETURN)

        sleep(3)

        wait(browser, 3, By.TAG_NAME, 'html')

        if 'nidlogin.login' in browser.current_url:
            print('로그인 실패')
            return None

        print('메일 서비스로 이동.')
        browser.get('http://mail.naver.com')

        wait(browser, 3, By.ID, 'list_for_view')

        print('page loaded !!!')

        html = browser.page_source
        soup = BeautifulSoup(html, 'html.parser')
        
        # blind 선제거
        for blind_tag in soup.select('.blind'):
            blind_tag.extract()
        
        for row_tag in soup.select('#list_for_view .mailList > li'):
            name_tag = row_tag.select('.name')[0]
            # name_tag.select('.blind')[0].extract()
            name = name_tag.text

            title_tag = row_tag.select('.subject .mail_title')[0]
            # title_tag.select('.blind')[0].extract()
            
            date_tag = row_tag.select('.iDate')[0]
            date = date_tag.text
            
            print(name, title_tag.text, date)


if __name__ == '__main__':
    user_id = input('Naver User ID : ')
    user_pw = getpass('Naver User Password : ')

    main(user_id, user_pw)

## Pillow

In [31]:
from PIL import Image
import requests

image = requests.get("https://ee5817f8e2e9a2e34042-3365e7f0719651e5b8d0979bce83c558.ssl.cf5.rackcdn.com/python.png").content
open("python3.png", "wb").write(image)

im = Image.open("python3.png")
print("current size : {}".format(im.size))
small_im = im.resize((300, 300))  # 새로운 Image인스턴스를 생성/리턴
# im.thumbnail((300, 300))  # 자신의 Image 인스턴스를 변경
small_im.save("python3_thumb.png")
small_im.save("python3_thumb.jpg")

canvas_white = Image.new('RGBA', small_im.size, (255, 255, 255))
merged_im = Image.alpha_composite(canvas_white, small_im)
merged_im.save('python3_bg_white.jpg')

current size : (1000, 1000)
